In [ ]:
import re
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data
import random
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns
from torch.utils.data import DataLoader, TensorDataset, Dataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_recall_curve, auc, roc_curve, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_class_weight
from collections import Counter
from itertools import product

In [ ]:
# List of TSV files to read
file_names = [
    '1ibis_QNZS_PBM13699.tsv', '1ibis_SD_PBM13699.tsv'
]

# Read each TSV file into a DataFrame and store in a list
dataframes = [pd.read_csv(file, sep='\t') for file in file_names]

# Assign each DataFrame to a variable
data1, data2 = dataframes

# Process QNZS-normalized PBMs
for df in [data1]:
    df['z_score'] = (df['mean_signal_intensity'] - df['mean_signal_intensity'].mean()) / df['mean_signal_intensity'].std()
    df['label'] = (df['z_score'] > 4).astype(int)

# Process SD-preprocessed PBMs
for df in [data2]:
    mean_intensity = df['mean_signal_intensity'].mean()
    std_dev_intensity = df['mean_signal_intensity'].std()
    threshold = mean_intensity + 4 * std_dev_intensity
    df['label'] = (df['mean_signal_intensity'] > threshold).astype(int)

# Iterate through the dataframes and print the sum count of label == 1
for i, df in enumerate(dataframes, 1):
    label_count = df['label'].sum()
    print(f"Sum count of label == 1 in data {i}: {label_count}")

# Concatenate two PBM datasets
concatenated_data = pd.concat([data1, data2])

# Extract two columns with aimed data and rename column as needed
concatenated_data = concatenated_data[['pbm_sequence', 'label']]
concatenated_data = concatenated_data.rename(columns={'pbm_sequence': 'sequence'})

In [ ]:
# Function to get the reverse complement of a DNA sequence
def reverse_complement(seq):
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    return ''.join(complement.get(base, base) for base in reversed(seq))

# Function to shift DNA sequences by a given number of positions
def shift_sequence(seq, shift):
    return seq[shift:] + seq[:shift]

# Function to perform nucleotide substitution at the first and last two positions
def mutate_edges(seq):
    bases = ['A', 'C', 'G', 'T']
    mutated_sequences = []
    # Generate all possible mutations for the first and last two positions
    for i1 in [0, 1]:
        for i2 in [-2, -1]:
            for _ in range(4):  # Attempt four mutations
                mutated_seq = list(seq)
                mutated_seq[i1] = random.choice([b for b in bases if b != seq[i1]])
                mutated_seq[i2] = random.choice([b for b in bases if b != seq[i2]])
                mutated_sequences.append(''.join(mutated_seq))
    return mutated_sequences


# Apply augmentation to positive cases
# Filter out the sequences where label = 1
label_1_sequences = concatenated_data[concatenated_data['label'] == 1]['sequence']

augmented_sequences = []
augmented_labels = []

# Apply augmentations
for seq in label_1_sequences:
    # Original sequence
    augmented_sequences.append(seq)
    augmented_labels.append(1)
    
    # Reverse complement
    rev_comp_seq = reverse_complement(seq)
    augmented_sequences.append(rev_comp_seq)
    augmented_labels.append(1)
    
    # Shift by 1 and 2 positions (both original and reverse complement)
    for shift in [1, 2]:
        shifted_seq_fwd = shift_sequence(seq, shift)
        augmented_sequences.append(shifted_seq_fwd)
        augmented_labels.append(1)
        
        shifted_seq_rev = shift_sequence(rev_comp_seq, shift)
        augmented_sequences.append(shifted_seq_rev)
        augmented_labels.append(1)
        
    # Mutate first and last two nucleotides in the original and reverse complement sequences
    mutated_seqs_original = mutate_edges(seq)
    mutated_seqs_rev_comp = mutate_edges(rev_comp_seq)
    
    for mutated_seq in mutated_seqs_original:
        augmented_sequences.append(mutated_seq)
        augmented_labels.append(1)
        
    for mutated_seq in mutated_seqs_rev_comp:
        augmented_sequences.append(mutated_seq)
        augmented_labels.append(1)

# Create a DataFrame for the augmented sequences
augmented_data = pd.DataFrame({
    'sequence': augmented_sequences,
    'label': augmented_labels
})

# Combine the augmented data with the original data_combined dataset
data_combined_augmented = pd.concat([concatenated_data, augmented_data], ignore_index=True)

print(f"Original class distribution:\n{concatenated_data['label'].value_counts()}")
print(f"Augmented class distribution:\n{data_combined_augmented['label'].value_counts()}")

In [ ]:
# Download datasets from HTS and SMS methods

sequences = []
labels = []

# Define file names and corresponding labels
fastq_files = [
    ('1_ibis_HTS_TIGD3_R0_C4_lf5ACGACGCTCTTCCGATCTGC_rf3CACTTCAGATCGGAAGAGCA.fastq', 1),
    # ('1_ibis_hts_LEF1_R0_C3_lf5ACGACGCTCTTCCGATCTAT_rf3AGCCTCAGATCGGAAGAGCA.fastq', 1), ('1_ibis_sms_TIGD3_UT380-225.fastq', 1)  # this two datasets were filtered out to more perfect model perfomance
]

# Define the minimum average quality score threshold
min_avg_quality = 37  # Adjust this threshold as needed

# Function to calculate the average quality score from a quality string
def calculate_avg_quality(quality_string):
    # Convert ASCII characters to Phred quality scores by subtracting 33
    return np.mean([ord(char) - 33 for char in quality_string])

# Loop through the files and extract sequences
for file_name, label in fastq_files:
    with open(file_name, 'r') as f:
        lines = f.readlines()

    for i in range(0, len(lines), 4):  # FASTQ format: 4 lines per sequence entry
        sequence = lines[i + 1].strip()  # The sequence is on the 2nd line of every 4-line entry
        quality = lines[i + 3].strip()   # The quality string is on the 4th line of every 4-line entry
        
        # Extract the last 40 base pairs of the sequence and their quality scores
        last_40_bp = sequence[-40:]
        last_40_quality = quality[-40:]

        # Calculate the average quality of the last 40 base pairs
        avg_quality = calculate_avg_quality(last_40_quality)
        
        # Check if the average quality meets the threshold
        if avg_quality >= min_avg_quality:
            sequences.append(last_40_bp)
            labels.append(label)  # Add the label for each sequence

# Create a DataFrame from the extracted sequences and labels
df_fastq = pd.DataFrame({
    'sequence': sequences,
    'label': labels
})

# Check if all sequences are of length 40
correct_length = all(len(seq) == 40 for seq in df_fastq['sequence'])
if correct_length:
    print("All sequences are of length 40.")
else:
    print("There are sequences that are not of length 40.")

# Check if all sequences contain only A, C, G, T
valid_bases = set('ACGT')
invalid_sequences = df_fastq['sequence'].apply(lambda seq: not set(seq).issubset(valid_bases))

# Report invalid sequences
if invalid_sequences.any():
    print(f"Found {invalid_sequences.sum()} sequences with invalid characters:")
    print(df_fastq[invalid_sequences])
else:
    print("All sequences contain only valid characters (A, C, G, T).")

# Concatenate the new DataFrame with df3
df3 = pd.concat([data_combined_augmented, df_fastq], ignore_index=True)

# Check the concatenated DataFrame
print(df3['label'].value_counts())

# # Filter out sequences that contain 'N' insteed of nucleotide
df3 = df3[~df3['sequence'].str.contains('N')]

# # Verify that there are no sequences with 'N'
assert df3['sequence'].str.contains('N').sum() == 0, "There are still sequences containing 'N'!"
df3['label'].value_counts()

In [ ]:
# Here, after fine tuning model with validation dataset, i train model in all dataset as train

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device available: {device}')

# Data preparation
X_dna = df3['sequence']
y = df3['label']

# Function to generate random DNA sequence based on the nucleotide distribution
def generate_random_dna(length, base_probabilities):
    bases = ['A', 'C', 'G', 'T']
    return ''.join(random.choices(bases, weights=base_probabilities, k=length))

# Function to compute nucleotide distribution
def compute_base_probabilities(sequences):
    total_len = sum(len(seq) for seq in sequences)
    base_counts = {base: sum(seq.count(base) for seq in sequences) for base in 'ACGT'}
    return [base_counts[base] / total_len for base in 'ACGT']

# Function to pad shorter sequences
def pad_sequences_with_random(sequences, max_len, opposite_base_probs):
    padded_sequences = []
    for seq in sequences:
        padding_length = max_len - len(seq)
        if padding_length > 0:
            random_seq = generate_random_dna(padding_length, opposite_base_probs)
            padded_seq = seq + random_seq
        else:
            padded_seq = seq
        padded_sequences.append(padded_seq)
    return padded_sequences

# Function to get the reverse complement of a DNA sequence
def reverse_complement(seq):
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    return ''.join(complement.get(base, base) for base in reversed(seq))

# Function to shift DNA sequences
def shift_sequence(seq, shift):
    return seq[shift:] + seq[:shift]

# Function to generate k-mers from a sequence
def generate_variable_kmers(sequence, k_sizes):
    kmers = []
    for k in k_sizes:
        kmers.extend([''.join(map(str, sequence[i:i+k])) for i in range(len(sequence) - k + 1)])
    return kmers

# Function to encode k-mers to integers
def integer_encode_variable_kmers(sequence, k_sizes):
    kmers = generate_variable_kmers(sequence, k_sizes)
    integer_encoded = [kmer_to_int[kmer] for kmer in kmers if kmer in kmer_to_int]
    return integer_encoded

# Augment sequences where y = 0 for training data
augmented_train_sequences = []
augmented_train_labels = []

shift_value = 3  # can be adjasted

for seq, label in zip(X_dna, y):
    if label == 0:
        # Apply reverse complement
        rev_comp_seq = reverse_complement(seq)
        augmented_train_sequences.append(rev_comp_seq)
        augmented_train_labels.append(0)
        
        # Apply shift sequence to the original and reverse complement sequences
        for i in range(1, shift_value + 1):  # Shift by 1 to shift_value positions
            shifted_seq_original = shift_sequence(seq, i)
            shifted_seq_rev_comp = shift_sequence(rev_comp_seq, i)
            
            # Store the shifted sequences and labels
            augmented_train_sequences.append(shifted_seq_original)
            augmented_train_sequences.append(shifted_seq_rev_comp)
            augmented_train_labels.append(0)
            augmented_train_labels.append(0)

# Combine the original sequences with augmented sequences
X_train_augmented = pd.Series(X_dna.tolist() + augmented_train_sequences)
y_train_augmented = pd.Series(y.tolist() + augmented_train_labels)

# Print the sizes of the original and augmented training datasets
print(f'The original train dataset size: {X_dna.shape[0]}')
print(f'The augmented train dataset size: {X_train_augmented.shape[0]}')

# Print the class distribution in the augmented training dataset
print(f'Class distribution in augmented train dataset:\n{y_train_augmented.value_counts()}')

# Calculate nucleotide probabilities from the negative dataset, this sequences will be used in padding
negative_sequences = [seq for seq in data_combined_augmented[data_combined_augmented['label'] == 0]['sequence']]  # as negatives can be choosen any class
negative_base_probs = compute_base_probabilities(negative_sequences)

# Padding sequences
max_len = max(len(seq) for seq in X_dna)
X_train_padded = pad_sequences_with_random(X_train_augmented, max_len, negative_base_probs)

# create k-mers - the DNA seq of len(k)
k_sizes = [2, 5]  # k-mer lengths can be optimised as you want
kmers = [''.join(p) for k in k_sizes for p in product('ACGT', repeat=k)]
kmer_to_int = {kmer: idx for idx, kmer in enumerate(kmers)}

# Encode padded sequences
X_train_encoded = [torch.tensor(integer_encode_variable_kmers(seq, k_sizes), dtype=torch.long) for seq in X_train_padded]

# Convert labels to tensors
y_train_tensor = torch.tensor(y_train_augmented.values, dtype=torch.float32)

# Create TensorDataset
train_dataset = TensorDataset(torch.stack(X_train_encoded), y_train_tensor)

# Set hyperparameters
num_embeddings = len(kmer_to_int)
embedding_dim = 130  
num_filters = 120
batch_size = 64
dropout_rate = 0.5
learning_rate = 0.001
num_epochs = 2

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# model architecture
class SimplifiedMaskedCNN(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, num_filters, dropout_rate):
        super(SimplifiedMaskedCNN, self).__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.conv1 = nn.Conv1d(embedding_dim, num_filters, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(num_filters, num_filters * 2, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        # self.conv3 = nn.Conv1d(num_filters * 2, num_filters * 4, kernel_size=3, padding=1)
        # self.pool3 = nn.MaxPool1d(kernel_size=2)
        # self.conv4 = nn.Conv1d(num_filters * 4, num_filters * 8, kernel_size=3, padding=1)
        # self.pool4 = nn.MaxPool1d(kernel_size=2)
        self.conv5 = nn.Conv1d(num_filters * 2, num_filters * 4, kernel_size=3, padding=1)
        self.pool5 = nn.AdaptiveAvgPool1d(1)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(4 * num_filters, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.embedding(x).transpose(1, 2)
        x = F.relu(self.pool1(self.conv1(x)))
        x = F.relu(self.pool2(self.conv2(x)))
        # x = F.relu(self.pool3(self.conv3(x)))
        # x = F.relu(self.pool4(self.conv4(x)))
        x = F.relu(self.pool5(self.conv5(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Focal Loss definition
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.75, gamma=2.5):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        targets = targets.unsqueeze(1)
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss
        return F_loss.mean()

# Instantiate the model
cnn = SimplifiedMaskedCNN(num_embeddings, embedding_dim, num_filters, dropout_rate).to(device)
criterion = FocalLoss()
optimizer = optim.AdamW(cnn.parameters(), lr=learning_rate, weight_decay=0.1)

# Initialize variables for metrics storage
train_losses = []
all_train_targets, all_train_scores = [], []

# Start training
for epoch in range(num_epochs):
    cnn.train()
    train_loss = 0
    train_targets, train_scores = [], []

    # Training loop
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        outputs = cnn(data)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * len(target)

        # Collect scores for metrics
        train_targets.extend(target.cpu().numpy())
        train_scores.extend(torch.sigmoid(outputs).detach().cpu().numpy().flatten())

    train_loss /= len(train_loader.dataset)
    train_losses.append(train_loss)
    all_train_targets.extend(train_targets)
    all_train_scores.extend(train_scores)

    # Calculate metrics
    train_precision, train_recall, _ = precision_recall_curve(train_targets, train_scores)
    train_auprc = auc(train_recall, train_precision)
    train_fpr, train_tpr, _ = roc_curve(train_targets, train_scores)
    train_auroc = auc(train_fpr, train_tpr)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, '
          f'Train AUPRC: {train_auprc:.4f}, Train AUROC: {train_auroc:.4f}')

    # Compute and display the confusion matrix for the current epoch
    y_pred = (np.array(train_scores) >= 0.5).astype(int)
    cm = confusion_matrix(train_targets, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.title(f'Confusion Matrix at Epoch {epoch+1}')
    plt.show()

# Save the model
model_path = 'cnn_model.pth'
torch.save(cnn.state_dict(), model_path)
print(f'Model saved to {model_path}')

# Save the model architecture
with open('cnn_model_architecture.pkl', 'wb') as f:
    pickle.dump(cnn, f)
print('Model architecture saved to cnn_model_architecture.pkl')



In [ ]:
from Bio import SeqIO

# for PBM
def parse_fasta(file_path):
    probe_ids = []
    sequences = []

    # Parse the FASTA file using SeqIO
    for record in SeqIO.parse(file_path, "fasta"):
        probe_id = record.id  # Extract the probe ID
        sequence = str(record.seq)  # Get the full sequence
        sequence_of_interest = sequence[-35:]  # Extract the last 35 nucleotides
        probe_ids.append(probe_id)
        sequences.append(sequence_of_interest)

    # Create a DataFrame
    df = pd.DataFrame({
        'probe_id': probe_ids,
        'sequence': sequences
    })

    return df

# Test data
file_path = "1_test_PBM_participants.fasta"  
parsed_data = parse_fasta(file_path)

Save the DataFrame to a CSV file for future use
parsed_data.to_csv("parsed_sequences.csv", index=False)


Load the parsed data from the CSV file
parsed_data = pd.read_csv("parsed_sequences.csv")
parsed_data

In [ ]:
# Data preparation
X_test_dna = parsed_data['sequence']
probe_ids = parsed_data['probe_id']  

# Encode the test sequences
X_test_encoded = [torch.tensor(integer_encode_variable_kmers(seq, k_sizes), dtype=torch.long) for seq in X_test_dna]

# Create a DataLoader for the test data
test_dataset = torch.utils.data.TensorDataset(torch.stack(X_test_encoded))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Make predictions
cnn.eval()
predictions = []

with torch.no_grad():
    for data in test_loader:
        data = data[0].to(device) 
        outputs = cnn(data)
        probs = torch.sigmoid(outputs).cpu().numpy().squeeze()  # Convert logits to probabilities
        predictions.extend(probs)

# Convert predictions to a DataFrame
predictions_df = pd.DataFrame({
    'probe_id': probe_ids,
    'score': predictions
})

# Ensure the predictions are within [0,1] and have up to 5 decimal places as needed according to IBIS requirements
predictions_df['score'] = predictions_df['score'].clip(0, 1).round(5)

print(predictions_df.head())